# Collision avoidance — drone

## Setup

Import modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import secrets
import ae353_drone
import ae353_dronecontrol           # <--- must have a working controller

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Create and print a seed for the random number generator so it is possible to reproduce the results.

In [ ]:
seed = secrets.randbits(32)
print(seed)

Create simulator with seed.

In [ ]:
simulator = ae353_drone.Simulator(seed=seed)

Add a camera view. This view will be displayed in its own browser window.

In [ ]:
simulator.add_view(
    'my_start_view',  # name of view (must be unique)
    'start',          # type of view (start, top, right, left, or back)
)

Allow print statements and long computation times (only for development).

In [ ]:
simulator.set_rules(
    error_on_print=False,
    error_on_timeout=False,
    error_on_inactive=False,
)

Show controller template. The actual controller used in this example comes from `ae353_dronecontrol.py`.

**NOTE. I didn't publish the version of `ae353_dronecontrol.py` that was used in class, rather just the template version. If you want this notebook to work, you'll need to modify `ae353_dronecontrol.py` so that it works!**

In [ ]:
class Controller:
    def __init__(self):
        # Define all variables (FIXME)
        #
        # self.A = ...
        # self.B = ...
        # ...
        # ...
        
        self.variables_to_log = ['xhat', 'xdes', 'ring']

    def get_color(self):
        return [1., 0., 0.]

    def reset(
            self,
            p_x, p_y, p_z, # <-- approximate initial position of drone (meters)
            yaw,           # <-- approximate initial yaw angle of drone (radians)
        ):
        
        # Choose initial state estimate (FIXME)
        # self.xhat = ...

    def run(
            self,
            pos_markers,
            pos_ring,
            dir_ring,
            is_last_ring,
            pos_others,
        ):
        
        # Store ring position for later analysis
        self.ring = np.array([p_x_ring, p_y_ring, p_z_ring])
        
        # Get xdes
        self.xdes = self.get_xdes(self.xhat, pos_ring, dir_ring, is_last_ring, pos_others)
        
        # Apply controller and observer... (FIXME)
        #
        # u = ...
        # y = ...
        # self.xhat += ...
        #
        # tau_x = ...
        # tau_y = ...
        # tau_z = ...
        # f_z = ...
        
        return tau_x, tau_y, tau_z, f_z
    
    def get_xdes(self, xhat, pos_ring, dir_ring, is_last_ring, pos_others):
        xdes = np.zeros(12)
        xdes[0:3] = np.array([0., 0., 1.])
        return xdes

Rearrange the rings.

In [ ]:
simulator.place_rings()

## Without collision avoidance

This controller is what we might start with before adding a planner with collision avoidance.

In [ ]:
class Controller(ae353_dronecontrol.Controller):
    def get_color(self):
        return [1., 0., 0.]
    
    def get_xdes(self, t, xhat, pos_ring, dir_ring, is_last_ring, pos_others):
        # Get estimated position
        phat = xhat[0:3]
        
        # Get desired position
        pdes = pos_ring
        
        # Make sure desired position is not too far away
        emax = 1.
        if np.linalg.norm(pdes - phat) > emax:
            pdes = phat + emax * ((pdes - phat) / np.linalg.norm(pdes - phat))
        
        # Get desired state
        xdes = np.zeros(12)
        xdes[0:3] = pdes
        return xdes

Clear all drones.

In [ ]:
simulator.clear_drones()

Add drones to simulator.

In [ ]:
num_drones = 5
for i in range(num_drones):
    simulator.add_drone(Controller, f'test_{i + 1}', 'template.png')

Reset the simulator.

In [ ]:
simulator.reset()

Run the simulator.

In [ ]:
simulator.run(max_time=25., print_debug=True)

## With collision avoidance

This controller is a template for adding a planner with collision avoidance.

In [ ]:
class AvoidanceController(ae353_dronecontrol.Controller):
    def get_color(self):
        return [0., 1., 0.]
    
    def get_xdes(self, t, xhat, pos_ring, dir_ring, is_last_ring, pos_others):
        # Get estimated position
        phat = xhat[0:3]
        
        ##############################
        
        # Define parameters
        k_att = 1.0
        k_rep = 1.0
        k_des = 1.0
        r_drone = 0.25  # <-- radius of a sphere that would completely contain each drone
        r_ring = 1.0    # <-- radius of each small ring
        
        # Get attractive part
        grad_h_att = np.zeros(3)            # <-- REPLACE WITH YOUR CODE
        
        # Get repulsive part
        # - Initialize the gradient
        grad_h_rep = np.zeros(3)
        # - Add repulsive part for each other drone
        for q in pos_others:
            grad_h_rep += np.zeros(3)       # <-- REPLACE WITH YOUR CODE
        # - Add repulsive part for next ring
        grad_h_rep += np.zeros(3)           # <-- REPLACE WITH YOUR CODE
        # - Add repulsive part for ground
        grad_h_rep += np.zeros(3)           # <-- REPLACE WITH YOUR CODE
        
        # Get desired position
        pdes = phat - k_des * (grad_h_att + grad_h_rep)
        
        ##############################
        
        # Make sure desired position is not too far away
        emax = 1.
        if np.linalg.norm(pdes - phat) > emax:
            pdes = phat + emax * ((pdes - phat) / np.linalg.norm(pdes - phat))
        
        # Get desired state
        xdes = np.zeros(12)
        xdes[0:3] = pdes
        return xdes

Clear all drones.

In [ ]:
simulator.clear_drones()

Add drones to simulator.

In [ ]:
num_drones = 5
for i in range(num_drones):
    simulator.add_drone(AvoidanceController, f'test_{i + 1}', 'template.png')

Reset the simulator.

In [ ]:
simulator.reset()

Run the simulator.

In [ ]:
simulator.run(max_time=5., print_debug=True)

### Show results

Get data.

In [ ]:
data = simulator.get_data('test_1')

Plot results (position only).

In [ ]:
p_x_est = data['xhat'][:, 0]
p_y_est = data['xhat'][:, 1]
p_z_est = data['xhat'][:, 2]

p_x_des = data['xdes'][:, 0]
p_y_des = data['xdes'][:, 1]
p_z_des = data['xdes'][:, 2]

p_x_ring = data['ring'][:, 0]
p_y_ring = data['ring'][:, 1]
p_z_ring = data['ring'][:, 2]

# Create a figure with subplots that all share the same x-axis
fig, (ax_px, ax_py, ax_pz) = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

ax_px.plot(data['t'], data['p_x'], label='x (m)', linewidth=3)
ax_px.plot(data['t'], p_x_est, '--', label='estimated x (m)', linewidth=4)
ax_px.plot(data['t'], p_x_des, ':', label='desired x (m)', linewidth=4)
ax_px.plot(data['t'], p_x_ring, '-.', label='ring x (m)', linewidth=3)
ax_px.grid()
ax_px.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_px.tick_params(labelsize=14)

ax_py.plot(data['t'], data['p_y'], label='y (m)', linewidth=3)
ax_py.plot(data['t'], p_y_est, '--', label='estimated y (m)', linewidth=4)
ax_py.plot(data['t'], p_y_des, ':', label='desired y (m)', linewidth=4)
ax_py.plot(data['t'], p_y_ring, '-.', label='ring y (m)', linewidth=3)
ax_py.grid()
ax_py.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_py.tick_params(labelsize=14)

ax_pz.plot(data['t'], data['p_z'], label='z (m)', linewidth=3)
ax_pz.plot(data['t'], p_z_est, '--', label='estimated z (m)', linewidth=4)
ax_pz.plot(data['t'], p_z_des, ':', label='desired z (m)', linewidth=4)
ax_pz.plot(data['t'], p_z_ring, '-.', label='ring z (m)', linewidth=3)
ax_pz.grid()
ax_pz.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_pz.tick_params(labelsize=14)

# Set shared x-axis properties
ax_pz.set_xlabel('time (s)', fontsize=20)
ax_pz.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()
plt.show()